In [17]:
# train
import pandas as pd

aws_cnt = 3
aws_codes = ['904', '941', '950']
aws_files = [f'train/{code}_interpolated.csv' for code in aws_codes]
drop_cols = ['일시', '지점']
aws = [pd.read_csv(aws_file).drop(drop_cols, axis=1).to_numpy().tolist() for aws_file in aws_files]
water = pd.read_csv('train/water_2022.csv').drop('관측 일시', axis=1).to_numpy().tolist()

X_aws, X_water, y_train = [[] for _ in range(aws_cnt)], [], []
num_features = 5
seq_len = 3
data_len = 7992
for i in range(num_features + seq_len - 1, data_len - seq_len):
    ie = i + seq_len
    for j in range(aws_cnt):
        X_aws[j].append(aws[j][i:ie])
    seq_wl = []
    for j in range(seq_len):
        wl = water[i - (num_features + seq_len) + 1 + j : i - seq_len + 1 + j]
        wl = [sublist[0] for sublist in wl]
        seq_wl.append(wl)
    X_water.append(seq_wl)
    seq_y = water[i:ie]
    seq_y = [sublist[0] for sublist in seq_y]
    y_train.append(seq_y)

In [21]:
# test
import pandas as pd

aws_cnt = 3
aws_codes = ['904', '941', '950']
aws_files = [f'test/{code}_interpolated.csv' for code in aws_codes]
drop_cols = ['일시', '지점', '지점명']
aws = [pd.read_csv(aws_file).drop(drop_cols, axis=1).to_numpy().tolist() for aws_file in aws_files]
water = pd.read_csv('test/water_2303_2308.csv').drop('ymdh', axis=1).to_numpy().tolist()

X_aws_test, X_water_test, y_test = [[] for _ in range(aws_cnt)], [], []
num_features = 5
seq_len = 3
data_len = 4344
for i in range(num_features + seq_len - 1, data_len - seq_len):
    ie = i + seq_len
    for j in range(aws_cnt):
        X_aws_test[j].append(aws[j][i:ie])
    seq_wl = []
    for j in range(seq_len):
        wl = water[i - (num_features + seq_len) + 1 + j : i - seq_len + 1 + j]
        wl = [sublist[0] for sublist in wl]
        seq_wl.append(wl)
    X_water_test.append(seq_wl)
    seq_y = water[i:ie]
    seq_y = [sublist[0] for sublist in seq_y]
    y_test.append(seq_y)

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, LSTM, Flatten, Dense, concatenate
from tensorflow.keras.models import Model

data_inputs = []
for i in range(aws_cnt):
    data_inputs.append(Input(shape=(seq_len, num_features), name=f'aws_input_{i}'))

data_inputs.append(Input(shape=(seq_len, num_features), name='water_input'))

merged = concatenate(data_inputs)

conv = Conv1D(32, kernel_size=3, activation='relu', padding='same')(merged)
conv = Conv1D(64, kernel_size=3, activation='relu', padding='same')(conv)

lstm = LSTM(50, return_sequences=True, name='lstm')(conv)
flatten = Flatten(name='flatten')(lstm)

# 출력 레이어
output = Dense(3, name='output')(flatten)

# 모델 생성
model = Model(inputs=data_inputs, outputs=output)
# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# 모델 요약 정보 출력
model.summary()

In [ ]:
import numpy as np

batch_size = 32
epochs = 1000

# X_aws와 X_water를 NumPy 배열로 변환
X_aws_numpy = np.array(X_aws, dtype=float)
X_water_numpy = np.array(X_water, dtype=float)

# y_train도 NumPy 배열로 변환
y_train_numpy = np.array(y_train)

# 모델 학습
model.fit([*X_aws_numpy, X_water_numpy], y_train_numpy, epochs=epochs, batch_size=batch_size)
model.save('model_1000.h5')

In [27]:
from tensorflow.keras.models import load_model
model = load_model('model_1000.h5')

In [ ]:
X_aws_test_numpy = np.array(X_aws_test, dtype=float)
X_water_test_numpy = np.array(X_water_test, dtype=float)
y_pred = model.predict([*X_aws_test_numpy, X_water_test_numpy])

from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

for i in range(seq_len):
    plt.figure(figsize=(10, 5))
    plt.plot(np.array(y_test)[:, i], label='Actual', color='red', alpha=0.5)
    plt.plot(y_pred[:, i], label='Predicted', color='blue', alpha=0.5)
    plt.title(f'Actual vs. Predicted {i + 1}h')
    plt.ylabel('Water Level')
    plt.legend()
    plt.show()

    mae = mean_absolute_error(np.array(y_test)[:, i], y_pred[:, i])
    mse = mean_squared_error(np.array(y_test)[:, i], y_pred[:, i])
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Root Mean Squared Error (RMSE): {np.sqrt(mse)}")